# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.65,78,92,12.29,TF,1691616214
1,1,punta arenas,-53.1500,-70.9167,3.73,91,100,3.51,CL,1691615864
2,2,olyokminsk,60.3736,120.4264,16.11,95,7,1.56,RU,1691616295
3,3,weno,7.4515,151.8468,26.97,100,75,2.06,FM,1691616228
4,4,chitapur,17.1167,77.0833,26.49,72,100,5.08,IN,1691616295


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    color="City",
    frame_width = 700,
    frame_height = 500
        
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp']>26) & (city_data_df['Max Temp']<33), :]
ideal_cities_df = ideal_cities_df.loc[(ideal_cities_df['Wind Speed']<10), :]
ideal_cities_df = ideal_cities_df.loc[(ideal_cities_df['Humidity']<60) & (ideal_cities_df['Humidity']>20), :]
ideal_cities_df = ideal_cities_df.loc[(ideal_cities_df['Cloudiness']<80) & (ideal_cities_df['Cloudiness']>20), :]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(how='any',axis=0)

# Display sample data
ideal_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,albany,42.6001,-73.9662,28.03,52,51,0.45,US,1691616128
40,40,yenotayevka,47.2456,47.0281,29.08,38,59,2.96,RU,1691616299
128,128,haiku-pauwela,20.9219,-156.3051,28.47,57,40,9.26,US,1691616313
209,209,hotan,37.0997,79.9269,27.68,26,55,2.90,CN,1691616324
235,235,turpan,42.9333,89.1667,32.59,27,30,1.55,CN,1691616328
284,284,jaguaribe,-5.8906,-38.6219,30.14,27,57,3.31,BR,1691616183
291,291,palmer,42.1584,-72.3287,28.65,53,75,5.14,US,1691616250
294,294,oulmes,33.4258,-6.0014,28.91,35,43,2.27,MA,1691616337
310,310,dubovskoye,47.4092,42.7575,26.40,52,36,3.66,RU,1691616339
479,479,emporia,38.4039,-96.1817,29.11,54,40,2.57,US,1691616168


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
36,albany,US,42.6001,-73.9662,52,
40,yenotayevka,RU,47.2456,47.0281,38,
128,haiku-pauwela,US,20.9219,-156.3051,57,
209,hotan,CN,37.0997,79.9269,26,
235,turpan,CN,42.9333,89.1667,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
yenotayevka - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
turpan - nearest hotel: Oriental
jaguaribe - nearest hotel: Hotel Pinheiro
palmer - nearest hotel: No hotel found
oulmes - nearest hotel: No hotel found
dubovskoye - nearest hotel: No hotel found
emporia - nearest hotel: Budget Host Inn Emporia
portland - nearest hotel: the Hoxton
sakaiminato - nearest hotel: かぐら旅館


,City,Country,Lat,Lng,Humidity,Hotel Name
36,albany,US,42.6001,-73.9662,52,No hotel found
40,yenotayevka,RU,47.2456,47.0281,38,No hotel found
128,haiku-pauwela,US,20.9219,-156.3051,57,Inn At Mama's Fish House
209,hotan,CN,37.0997,79.9269,26,Hua Yi International Hotel (accepts foreigners)
235,turpan,CN,42.9333,89.1667,27,Oriental
284,jaguaribe,BR,-5.8906,-38.6219,27,Hotel Pinheiro
291,palmer,US,42.1584,-72.3287,53,No hotel found
294,oulmes,MA,33.4258,-6.0014,35,No hotel found
310,dubovskoye,RU,47.4092,42.7575,52,No hotel found
479,emporia,US,38.4039,-96.1817,54,Budget Host Inn Emporia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    size="Humidity",
    hover_cols=["Country", "Hotel Name"],
    color="City",
    frame_width = 700,
    frame_height = 500    
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)